# 🎶 AI Quartet Pitch Comparison
This notebook compares a student's singing recording with a professor's reference using pitch analysis and waveform overlay.
**Note:** Just run the cells below. No need to paste anything — the audio URLs are passed dynamically from Zapier.

In [ ]:
# Install required packages
!pip install librosa matplotlib numpy requests

In [ ]:
# Import libraries
import librosa
import matplotlib.pyplot as plt
import numpy as np
import requests
from IPython.display import Audio
from urllib.parse import urlparse, parse_qs
import IPython.display as display

In [ ]:
# Extract URLs from Colab URL
parsed_url = urlparse(notebook_url)
params = parse_qs(parsed_url.query)
professor_url = params.get('professor_url', [None])[0]
student_url = params.get('student_url', [None])[0]
print('Professor URL:', professor_url)
print('Student URL:', student_url)

In [ ]:
# Define audio loader
def load_audio_from_url(url):
    r = requests.get(url)
    with open('temp.mp3', 'wb') as f:
        f.write(r.content)
    audio, sr = librosa.load('temp.mp3')
    return audio, sr

In [ ]:
# Load audio
prof_audio, sr1 = load_audio_from_url(professor_url)
stud_audio, sr2 = load_audio_from_url(student_url)
if sr1 != sr2:
    stud_audio = librosa.resample(stud_audio, orig_sr=sr2, target_sr=sr1)
    sr2 = sr1

In [ ]:
# Extract pitch
prof_pitch, _ = librosa.piptrack(y=prof_audio, sr=sr1)
stud_pitch, _ = librosa.piptrack(y=stud_audio, sr=sr2)
prof_pitch_mean = np.mean(prof_pitch, axis=0)
stud_pitch_mean = np.mean(stud_pitch, axis=0)

In [ ]:
# Plot pitch comparison
plt.figure(figsize=(12, 4))
plt.plot(prof_pitch_mean, label='Professor')
plt.plot(stud_pitch_mean, label='Student')
plt.title('Pitch Comparison')
plt.xlabel('Frame')
plt.ylabel('Pitch (Hz)')
plt.legend()
plt.grid(True)
plt.show()